# 2 Tabular Reinforcement Learning
In this notebook we are going to explore algorithms with which we can train agents to solve more or less simple environments on their own. Specifically, we will have a look at the SARSA algorithm. Instead of telling the agents what to do explicitely, we will only punish and reward their actions. Thereby, the agents should figure out a strategy by themselves, perhaps in ways that we would not have foreseen.

Before we can start working on our agents, we have to set up an environment. The environment in a reinforcement learning problem is the world in which the agent acts. Games are very good example environments because they are sufficiently simple and can be simulated easily.

## 2.1 The environment
We will make use of the [gym](https://gym.openai.com/) library from openAI. It provides a number of environments with varying degrees of complexity in a convenient form.

In [ ]:
!pip install gym

In [ ]:
# import stuff
import gym
from IPython import display
import time
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1337)

In our task, we want to use the environment [Taxi-v3](https://gym.openai.com/envs/Taxi-v3/). Let's have a look (because we are using Jupyter notebooks, we need to use the supplied function below to render the environment).

In [ ]:
def nb_render(env):
    # render gym environments in jupyter notebooks
    display.clear_output(wait=True)
    env.render()
    time.sleep(0.05)

In [ ]:
env = gym.make('Taxi-v3')
env.reset() # resets the environment
nb_render(env)

In Taxi-V3, the agent is a taxi driver (yellow if empty, green if full) in a 5x5 square. The aim is to pick up a passenger (at the location in bold blue font) and drop the passenger off at the target location (in magenta).

__Rewards__

A successful drop-off gives a reward of 20. 
Attempting to pick-up or drop-off prematurely gives a negative reward (punishment) of -10.
Each step taken also gives a negative reward of -1.

__Actions__

In [ ]:
print(env.action_space.n)

The agent can execute one of six actions. Move north, south, east, west, pick-up or drop-off.

__State-space__

(States are called observations in gym lingo)

We can encounter 500 different states in this environment (25 taxi positions * 5 passenger locations * 4 destination locations).

In [ ]:
print(env.observation_space.n)

__Interaction__

Interacting with the environment is really straight-forward. Simply supply the index of an action to the step-function of the environment.
As return values, you will receive the next state, the reward for the action and if the environment is solved (you can ignore the fourth return value).

In [ ]:
s, r, done, _ = env.step(3)
nb_render(env)

## 2.2 Evaluation of an agent
Since we want to have a _successful_ agent, we need a measure to evaluate that success. The cumulative reward over the whole episode should suffice for that purpose. An episode is the whole interaction process between agent and environment until the problem is solved or otherwise interrupted.

### Task 2.2.1 Random agent
In order to test our evaluation and compare our agent later, we need to have a baseline agent.
Since we do not know any better at the moment, let's implement an agent that acts according to a random policy.
Write a function _random\_policy()_ that takes the current state as input and returns a random action.

In [ ]:
def random_policy(state):
    # your code here

### Task 2.2.1 Evaluation function
Write a function _evaluate()_ that takes a policy function as input, runs the policy on 20 independent episodes of the environment and returns a list of the cumulative rewards of those episodes.

_Hint_: The _reset()_ function should be run before each episode starts and provides you with the first state.

In order to see what our agent is doing, render the last episode. 

In [ ]:
def evaluate(policy):
    # your code here

### Task 2.2.3 Evalute the random agent
Run the evaluation scheme on the random agent and plot the rewards.

In [ ]:
# your code here

In [ ]:
# your code here

## 2.3 SARSA
Now we want to write an agent that learns from rewards and in the end performs the task a little better.
From the lecture, we can remember that the SARSA algorithm deals with Q values. These values tell us how good it is to take a specific action in a specific state. For descrete problems up to a certain complexity (as is the Taxi-v3) we can simply store the Q values in a table.

### Task 2.3.1 The Q-Table
Write the function _init\_table()_ that sets up and returns a table that contains the Q values of state-action pairs.
Initialise all Q values with zeros.
Over time, while learning, the agent will change these values in order to prefer certain actions in certain situations.


In [ ]:
def init_table():
    # your code here

### Task 2.3.2 Exploitation: greedy strategy
The Q table represents our knowledge about the goodness of actions. Usually, we want to choose the best action for a certain situation. This way of choosing actions can be seen as exploiting your knowledge.
Write a function _exploit()_ that returns the best action for a given state according to your Q-table.

In [ ]:
def exploit(state):
    # your code here

### Task 2.3.3 Exploitation vs. Exploration
However, greedily executing actions all the time is not the best idea. A big problem in reinforcement learning is sufficent exploration of the environment. You will never know if your current strategy is optimal, if you did not fully explore all possibilities. But exploring is costly, as it means to deviate from exploiting your current strategy.

One way to tackle this dilemma is the $\epsilon$-greedy strategy. For a certain fraction of actions $\epsilon$, we will not use our knowledge to choose the best action, but rather choose a random non-greedy action to explore the environment.

Write a function _explore()_ that returns a random non-greedy action for a given state according to your Q-table.

In [ ]:
def explore(state):
    # your code here

Write a function _eps\_greedy()_ that takes a state as input. In $\epsilon$ of all cases return an explorative action and in 1-$\epsilon$ of all cases return an exploitative action.

In [ ]:
epsilon = 0.1

def eps_greedy(state):
    # your code here

### Task 2.3.1 The temporal difference error
We only need one last piece to complete our algorithm: updating our knowledge about good state-action combinations. Hence, we have to update our Q-values in a way that reflects how good an action is to reach the goal in the future.

The TD-error for SARSA is given as (see lecture p.19):

$$ \delta =  r + \gamma Q(s';a') - Q(s;a) $$

Write a function _td\_error()_, that takes as arguments the current state, the taken action, the reward, the next state and the next action and returns the temporal difference error

In [ ]:
gamma = 0.99

def td_error(s, a, r, s_prime, a_prime):
    # your code here

Lastly, the Q table needs to be updated according to:
    $$ Q(s;a) \leftarrow Q(s;a) + \eta \delta $$
    
Write a function _update\_table()_ that takes as inputs the current state, the current action and the td-error and updates the Q table.

In [ ]:
eta = 0.1

def update_table(s, a, delta):
    # your code here

## 2.4 Training
Now we can finally train our agent. 2000 episodes should suffice to sufficiently solve the Taxi problem.

The SARSA algorithm for one episode goes as follows:

- initialise environment, read initial state s
- select an action a
- repeat until done:
    - execute action $a$
    - read reward $r$ and new state $s'$
    - select next action $a'$
    - compute TD-error
    - update Q table
    - set variables for next iteration: 
        $s \leftarrow s'$, $a \leftarrow a'$

### Task 2.4.1 The train function
Write a train function that applies the SARSA algorithm to the environment for 2000 episodes. The function should return a list of cumulative rewards for each episode.

In [ ]:
table = init_table()
episodes = 2000

def train():
    # your code here

train_rewards = train()

### Task 2.4.2 Visualise the training
Train your agent and obtain the cumulative rewards for the episodes.
Plot the temporal evolution of the rewards over the episodes in order to see if the training worked.

In [ ]:
# your code here

## 2.5 Results
Now that we have a trained agent it is time to evaluate the trained agent and compare it against the performance of the random agent.

### Task 2.5.1 Evaluate the trained agent
We already wrote the evaluation function. So now we just need to apply our policy to the environment and see how the agent performs.
It is perhaps best to just use the greedy policy. If we assume that our agent is fully trained to the best of our knowledge, and we are not updating our Q table and thus are not learning during evaluation, there is no need for exploration.

In [ ]:
# your code here

### Task 2.5.2 Compare the agents
Plot the results of the random agent vs the results of the trained agent for a nice comparison.

In [ ]:
# your code here